In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

file = 'FuelConsumptionCo2.xlsx'
df = pd.read_excel(file)

# Eliminar columnas categóricas
df = df.drop(columns=['MAKE', 'MODEL', 'VEHICLECLASS', 'TRANSMISSION', 'FUELTYPE'])
df.head()

,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2022,2.4,4,9.9,7.0,8.6,33,200
1,2022,3.5,6,12.6,9.4,11.2,25,263
2,2022,2.0,4,11.0,8.6,9.9,29,232
3,2022,2.0,4,11.3,9.1,10.3,27,242
4,2022,2.0,4,11.2,8.0,9.8,29,230


In [33]:
# Separar variables predictoras (X) y variable objetivo (y)
X = df.drop(columns=['CO2EMISSIONS'])
y = df['CO2EMISSIONS']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [63]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
# Calcular los indicadores de bondad de ajuste
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5

print(f"R²: {r2}")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")


R²: 0.9004924315633444
MAE: 7.339144920646459
RMSE: 19.20591732975202


In [53]:
# Coeficientes del modelo
coef = model.coef_
intr = model.intercept_
print("Coeficientes:", coef)
print("Intercepto:", intr)

Coeficientes: [ 0.          0.22905536  4.06425593  1.21642426  3.52221659 12.66870885
 -1.25823447]
Intercepto: 81.47847459719395


In [55]:
# Predicción manual vs predicción del modelo
fir_obs = X_test.iloc[0]
manual_prediction = intr + sum(fir_obs.values * coef)
model_prediction = model.predict([fir_obs.values])[0]

print(f"Predicción manual: {manual_prediction}")
print(f"Predicción del modelo: {model_prediction}")
print(f"Diferencia absoluta: {abs(manual_prediction - model_prediction)}")

Predicción manual: 242.92694037393744
Predicción del modelo: 242.92694037393744
Diferencia absoluta: 0.0


# Coincide el resultado con el predict, al final tenemos que es el mismor resultado, ya que están usando los mismos coeficientes y el mismo intercepto, y el cálculo matemático es idéntico.

# RIDGE


In [98]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV


# Definir valores de alpha
alpha_ran = {'alpha': [0.01, 0.1, 1, 10, 100]}

In [100]:
# Busqueda de rango de alpha y entrenamiento para Rigde
ridge_search = GridSearchCV(Ridge(), alpha_ran, cv=5, scoring='r2')
ridge_search.fit(X_train, y_train)
ridge_model = ridge_search.best_estimator_

In [104]:
# Predicción con Ridge
y_pred_ridge = ridge_model.predict(X_test)
r2_ridge = r2_score(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
rmse_ridge = mean_squared_error(y_test, y_pred_ridge) ** 0.5

print("Modelo Ridge:")
print(f"Mejor alpha: {ridge_search.best_params_['alpha']}")
print(f"R²: {r2_ridge}")
print(f"MAE: {mae_ridge}")
print(f"RMSE: {rmse_ridge}")

Modelo Ridge:
Mejor alpha: 100
R²: 0.9038571672241419
MAE: 7.409746948807305
RMSE: 18.878411799701507


In [114]:
fir_obsR = X_test.iloc[0]
manual_prediction_ridge = ridge_model.intercept_ + sum(fir_obsR.values * ridge_model.coef_)
model_prediction_ridge = ridge_model.predict([fir_obsR.values])[0]

print(f"Predicción manual (Ridge): {manual_prediction_ridge}")
print(f"Predicción del modelo (Ridge): {model_prediction_ridge}")
print(f"Diferencia absoluta: {abs(manual_prediction_ridge - model_prediction_ridge)}")

Predicción manual (Ridge): 243.9490513916955
Predicción del modelo (Ridge): 243.94905139169552
Diferencia absoluta: 2.842170943040401e-14


# En el uso de modelo Ridge como tienda a regularizar los datos ayuda a reducir el sobreajuste y tiene mejoria en la precision de los datos. Aunque entre la predicion manual y el de Ridge es una diferencia muy pequeña si se eliminan valores decimal, seria el mismo resultado

# Lasso


In [120]:
# Busqueda de rango de alpha y entrenamiento para Lasso
lasso_search = GridSearchCV(Lasso(), alpha_ran, cv=5, scoring='r2')
lasso_search.fit(X_train, y_train)
lasso_model = lasso_search.best_estimator_

In [124]:
# Predicción con Lasso
y_pred_lasso = lasso_model.predict(X_test)
r2_lasso = r2_score(y_test, y_pred_lasso)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
rmse_lasso = mean_squared_error(y_test, y_pred_lasso) ** 0.5

print("\nModelo Lasso:")
print(f"Mejor alpha: {lasso_search.best_params_['alpha']}")
print(f"R²: {r2_lasso}")
print(f"MAE: {mae_lasso}")
print(f"RMSE: {rmse_lasso}")



Modelo Lasso:
Mejor alpha: 1
R²: 0.9023948843500449
MAE: 7.3594421856023065
RMSE: 19.021435473733696


In [128]:
# Realizar la predicción para la primera observación en el conjunto de prueba
first_obsL = X_test.iloc[0]
manual_prediction_lasso = lasso_model.intercept_ + sum(first_obsL.values * lasso_model.coef_)
model_prediction_lasso = lasso_model.predict([first_obsL.values])[0]

# Mostrar resultados
print(f"Predicción manual (Lasso): {manual_prediction_lasso}")
print(f"Predicción del modelo (Lasso): {model_prediction_lasso}")
print(f"Diferencia absoluta: {abs(manual_prediction_lasso - model_prediction_lasso)}")

Predicción manual (Lasso): 244.59118444593796
Predicción del modelo (Lasso): 244.59118444593796
Diferencia absoluta: 0.0


# La diferencia entre la prediccion manual y la prediccion es nula, ya que en esta ocasion Lasso puede llegar a descartar datos o eliminar si tienden a 0. tiende a dar igual por los coeficientes que se usan para su ajuste

# Para este caso escogeria una el modelo lineal de regresion ya que el resultado de cada modelo es muy similar, no siento la necesidad los otros dos modelos,  tenemos datos completos y limpios, podria llegar a usar el modelo de Ridge o Lasso si hubiese una diferencia mas grande entre datos y con ellos dependiendo el resultado de las predicciones,elijiria el que tuviera mejor resultado